## Circular, Array-back Queue

The backing array will be created and populated with Nones in the __init__ method, and the head and tail indexes set to sentinel values. Enqueueing and Dequeueing items will take place at the tail and head, with tail and head tracking the position of the most recently enqueued item and that of the next item to dequeue, respectively. To simplify testing, your implementation should make sure that when dequeuing an item its slot in the array is reset to None, and when the queue is emptied its head and tail attributes should be set to -1.

Because of the fixed size backing array, the enqueue operation is defined to raise a RuntimeError when the queue is full — the same exception should be raised when dequeue is called on an empty queue.

Finally, the resize method will allow the array underlying the queue to be increased in size. It is up to you how to implement this (you can either leave the elements in their current positions, though this may require "unwrapping" elements, or you can simply move all elements towards the front of the array). You may assume that resize will only be called with a value greater than the current length of the underlying array.

In [20]:
class Queue:
    def __init__(self, limit=10):
        self.data = [None] * limit
        self.head = -1
        self.tail = -1        

        
    def enqueue(self, val):
        if self._full():
            raise RuntimeError
        elif self.empty():
            self.head = self._increment(self.head)
            self.tail = self._increment(self.tail)
            self.data[self.head] = self.data[self.tail] = val
        else:
            self.tail  = self._increment(self.tail)
            self.data[self.tail] = val
                    
            
    def dequeue(self):
        if self.empty():
            raise RuntimeError
        item_to_remove = self.data[self.head]
        self.data[self.head] = None
        if self.empty():
            self.head = self.tail = -1
        else:
            self.head = self._increment(self.head)
        return item_to_remove
        
    
    def resize(self, new_size):
        assert new_size > len(self.data)
        new_data = [None] * 10
        old_head = self.head
        old_tail = self.tail
        for i in range(len(self.data)):
            new_data[i] = self.data[old_head]
            old_head = self._increment(old_head)
        self.data = new_data
        self.head = 0
        self.tail = old_tail + 1
  
        
    def empty(self):
        return self.data.count(None) == len(self.data)
    
    
    def _full(self):
        return self.data.count(None) == 0
    
    
    def _increment(self, index):
        return (index + 1) % len(self.data)
    
    
    def __bool__(self):
        return not self.empty()
    
    
    def __str__(self):
        return '<--'.join(str(x) for x in self.__iter__())
    
    
    def __repr__(self):
        str(self.data)
        
    
    def __iter__(self):
        for element in self.data:
            if element != None:
                yield self.data[self.head]
            self.head = (self.head + 1) % len(self.data)             

In [21]:
from unittest import TestCase
tc = TestCase()

q = Queue(5)
tc.assertEqual(q.data, [None] * 5)

for i in range(5):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(5)

for i in range(5):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())

In [22]:
from unittest import TestCase
tc = TestCase()

q = Queue(10)

for i in range(6):
    q.enqueue(i)
    
tc.assertEqual(q.data.count(None), 4)

for i in range(5):
    q.dequeue()
    
tc.assertFalse(q.empty())
tc.assertEqual(q.data.count(None), 9)
tc.assertEqual(q.head, q.tail)
tc.assertEqual(q.head, 5)

for i in range(9):
    q.enqueue(i)

with tc.assertRaises(RuntimeError):
    q.enqueue(10)

for x, y in zip(q, [5] + list(range(9))):
    tc.assertEqual(x, y)
    
tc.assertEqual(q.dequeue(), 5)
for i in range(9):
    tc.assertEqual(q.dequeue(), i)

tc.assertTrue(q.empty())

In [23]:
from unittest import TestCase
tc = TestCase()

q = Queue(5)
for i in range(5):
    q.enqueue(i)

for i in range(4):
    q.dequeue()

for i in range(5, 9):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(10)

q.resize(10)

for x, y in zip(q, range(4, 9)):
    tc.assertEqual(x, y)

for i in range(9, 14):
    q.enqueue(i)

for i in range(4, 14):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())
tc.assertEqual(q.head, -1)